In [ ]:
import openai
import json
from openai import OpenAI
import os
from tqdm import tqdm

# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = xxx
client = OpenAI()

def check_match_with_gpt(question, ground_truth, predicted):
    # Construct the prompt for GPT-4
    prompt = f"Question: {question}\nGround Truth Answer: {ground_truth}\nPredicted Answer: {predicted}\nDoes the predicted answer match the ground truth? Answer 1 for match and 0 for not match. Use semantic meaning not exact match. Synonyms are also treated as a match, e.g., football and soccer, playground and ground track field, building and rooftop, pond and swimming pool. Do not explain the reason.\n"

    # "If the ground truth is included in the predicted answer, it is considered a match, such as a viaduct belongs to the highway system, commercial belongs to ubran, villa belongs to residential." 

    response = client.chat.completions.create(
        # model="gpt-3.5-turbo-1106",
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text", 
                        "text": prompt,
                    },
                ]
            }
        ],
        max_tokens=100,
    )

    # answer = response.choices[0].text.strip()
    answer =  response.choices[0].message.content
    
    return answer



# VRSBench

In [13]:
# qa_list = [json.loads(line) for line in open('../r1-v/outputs/VRSBench/eval/pred_s200.json','r').readlines()]
qa_list = json.load(open('../r1-v/outputs/VRSBench/eval/pred_s500.json','r'))['results']

# Iterate over the list and check matches
results = []
f = open('../r1-v/outputs/VRSBench/eval/pred_s500_gpt.json', 'a') 
for ii, qa in enumerate(tqdm(qa_list)):
    question = qa['question']
    ground_truth = qa['ground_truth'].lower()
    predicted = qa['extracted_answer'].lower()
    if ground_truth in predicted:
        match_result = '1'
    elif ground_truth in ['yes', 'no'] + list(map(str, range(100))):
        match_result = '1' if ground_truth == predicted else '0'
    elif 'correct' not in qa or qa['correct'] not in ['1', '0']:
        # try to call gpt, if got error, sleep for 1 hour
        try:
            match_result = check_match_with_gpt(question, ground_truth, predicted)
        except:
            import time
            time.sleep(3600)
            match_result = check_match_with_gpt(question, ground_truth, predicted)
    else:
        match_result = qa['correct']
        
    result = {
        'question_id': qa['question_id'],
        'image_id': qa['image_id'],
        "type": qa['type'],
        "question": question,
        "ground_truth": ground_truth,
        "predicted": predicted,
        "correct": match_result,
    }
    results.append(result)

    f.write(json.dumps(result)+'\n')
    f.flush()

f.close()



  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [02:25<00:00,  3.43it/s]


In [14]:
f = open('../r1-v/outputs/VRSBench/eval/pred_s500_gpt.json', 'r') 
results = [json.loads(line) for line in f.readlines()]
f.close()
correct = sum([int(result['correct']) for result in results if result['correct'] in ['1', '0']])
print(f"Correct: {correct}/{len(results)}:", correct/len(results))

Correct: 279/500: 0.558
